# SQL and Pandas fun using rdkit generated structures

This notebook is inspired by the work of Elholm et al. (2022) DOI: 10.1039/d2cp03032b .
The background on clustering was based on Greg Landrum's work as well as the Liam Wilbraham for substituent generation.
The code for python to sql conversions was taken from the tutorial by Craig Dickson and uses an MySQL Server.

More Scientific Background: doi: 10.1021/acs.jcim.1c00256. Epub 2021 Jun 23


## installation of required packages

In [ ]:
### installation of chemical libraries:
# for rdkit see: https://www.rdkit.org/docs/GettingStartedInPython.html
# for ase  see: https://wiki.fysik.dtu.dk/ase/index.html
# further inspirations: https://peterschindler.github.io/


#! conda update -n base -c defaults conda
#! conda create -n rdkit -y
! conda activate rdkit 
#! conda install -c conda-forge rdkit -y 

#! pip install --upgrade --user ase
#! pip install scipy --upgrade --user
#! pip install --upgrade pandas "dask[complete]"
#! pip install --user smilescombine
#! conda install matplotlib
#! pip install py3Dmol

#conda deactivate


In [ ]:
### installation of sql requirements

# see: https://www.freecodecamp.org/news/connect-python-with-sql/
# and: https://realpython.com/python-sql-libraries/

#! pip install mysql-connector-python
#! pip install pandas

In [ ]:
#! conda deactivate

## Let's start with defining the SQL/Python commands and connection

In [ ]:
### connecting to server

import mysql.connector
from mysql.connector import Error
import pandas as pd

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

pw = 

connection = create_server_connection("localhost", "root", pw)


In [ ]:
### creating the database

import mysql.connector
from mysql.connector import Error
import pandas as pd

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
create_database_query= 'CREATE DATABASE substituents'
create_database(connection, create_database_query)


In [ ]:
### creating connection to specific database

import mysql.connector
from mysql.connector import Error
import pandas as pd

def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [ ]:
### creating a query execution function

import mysql.connector
from mysql.connector import Error
import pandas as pd

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

## Let's create the SQL library

Architecture of simple database:


In [ ]:
### create the substituents table

import mysql.connector
from mysql.connector import Error
import pandas as pd

create_substituent_table = """
CREATE TABLE substituents (
  substituent_id CHAR PRIMARY KEY,
  smiles VARCHAR(50) NOT NULL,
  atomseq VARCHAR(50) NOT NULL
  numconf VARCHAR(3) NOT NULL
  numclust VARCHAR(3) NOT NULL,
  lowest_energy VARCHAR(20) NOT NULL,
  );
 """

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database
execute_query(connection, create_teacher_table) # Execute our defined query

In [ ]:
### create the conformers table

import mysql.connector
from mysql.connector import Error
import pandas as pd

create_substituent_table = """
CREATE TABLE conformers (
  substituent_id CHAR PRIMARY KEY,
  atoms VARCHAR(50) NOT NULL,
  positions VARCHAR(100) NOT NULL,
  cluster_no VARCHAR(3) NOT NULL,
  conformer_num VARCHAR(4) NOT NULL,
  energies VARCHAR(20) NOT NULL
  );
 """

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database
execute_query(connection, create_teacher_table) # Execute our defined query

In [ ]:
### create the clusters table

import mysql.connector
from mysql.connector import Error
import pandas as pd

create_clusters_table = """
CREATE TABLE clusters (
  substituent_id CHAR PRIMARY KEY,
  clustID VARCHAR(50) NOT NULL,
  clust_le VARCHAR(100) NOT NULL,
  clust_ae VARCHAR(3) NOT NULL,
  clust_me VARCHAR(4) NOT NULL,
  );
 """

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db) # Connect to the Database
execute_query(connection, create_teacher_table) # Execute our defined query

In [ ]:
### additionally one could define the relationships between tables but we won't do it here

## Define commands how to populate the table and correct table set-up

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [ ]:
#! conda deactivate

## Let's continue with the actual Chemistry in rdkit

In [ ]:
## Construct substituent library according to: https://github.com/LiamWilbraham/smilescombine
# alternative way: https://iwatobipen.wordpress.com/2021/06/06/generate-all-combinations-of-rgroups-from-molecules-rdkit-chemoinformatics/
# smile of nbd reads by the way as:
# NBD = Cc1cc(C)cc(C)c1C2=CC3C=CC2C3

from smilescombine import Combiner
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import numpy as np
import pandas as pd

IPythonConsole.drawOptions.minFontSize = 10
IPythonConsole.molSize = 350,300

# m = Chem.MolFromSmiles('Cc1cc(C)cc(C)c1C2=CC3C=CC2C3')
# s = Chem.MolToSmiles(m)
# print(s) #-> Cc1cc(C)c(C2=CC3C=CC2C3)c(C)c1 ; C(Br)c1cc(C)c(C2=C(Br)C3C=CC2C3)c(C)(Br)c1

substituents = ['(CN)', '(N(=O)(=O))', '(CO)', '(Cl)', '(F)', '(C(=C(CN)(CN)))', '(OCC)', '(C)']

skeleton = Combiner('C(Br)c1cc(C(Br))c(C2=C(Br)C3C=CC2C3)c(C(Br))c1', substituents, nmax=4, nconnect=0, auto_placement=False)
skeleton.combine_substituents()

# extract smiles into pandas data frame
skeleton2 = skeleton.combinations 
initial_df = pd.DataFrame({'smiles':skeleton2})

# add id's
initial_df['num'] = np.arange(initial_df.shape[0])

def onesizenum(num):
    padding = 4
    str(num).zfill(padding)
    return num

initial_df['num_l'] = inizial_df.num.apply(onesizenum)

initial_df["id_list"] = initial_df['num_l'].astype(str) +"_"+"substituent"
initial_df = initial_df.drop('num_l', axis=1)
initial_df = initial_df.drop('num', axis=1)

# convert with numpy
columns = initial_df[["id_list", "smiles"]]
arr1 = [tuple(r) for r in columns.to_numpy()]



In [ ]:
### insert smiles and ID's into substituent table

sql_1 = '''
    INSERT INTO substituents (substituent_id, smiles) 
    VALUES (%s, %s)
    '''
    
val_1 = arr1

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql_1, val_1)


In [ ]:
## generate 3D conformers
# from: https://greglandrum.github.io/rdkit-blog/posts/2023-02-04-working-with-conformers.html
# and: https://patwalters.github.io/practicalcheminformatics/
# scientific background: https://rdkit.org/UGM/2012/Ebejer_20110926_RDKit_1stUGM.pdf
# this can take a while -> for 50 conformers (3645 substituents) it took 50 minutes 

import rdkit
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
#import py3Dmol
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom

import pandas
from tqdm.notebook import tqdm
tqdm.pandas()


def generateconformations(m):
    mol = Chem.MolFromSmiles(m)
    mol = Chem.AddHs(mol)
    ids=AllChem.EmbedMultipleConfs(mol, numConfs=numConfs, maxAttempts=maxAttempts, useRandomCoords=useRandomCoords, pruneRmsThresh=pruneRmsThresh, useExpTorsionAnglePrefs=useExpTorsionAnglePrefs, useBasicKnowledge=useBasicKnowledge, enforceChirality=enforceChirality, numThreads=4)
        return mol

# these variables need to be adjusted for molecule for which conformers are to be generated
# for example: https://github.com/rdkit/rdkit/discussions/5841
# variables can be looked up here: https://www.rdkit.org/docs/source/rdkit.Chem.rdDistGeom.html
numConfs = 50
maxAttempts = 10
pruneRmsThresh = 0.5
useExpTorsionAnglePrefs = True
useBasicKnowledge = False
enforceChirality = True
useRandomCoords = True

# in case, you want to test it on a subset first:
df_sh = initial_df.sample(frac =1)

df_sh['confMOL'] = df_sh.smiles.progress_apply(generateconformations)



In [ ]:
### check how many conformers were generated
# alternative program to generate conformers: https://open-babel.readthedocs.io/en/latest/3DStructureGen/multipleconformers.html
# see for alternative: https://github.com/rdkit/rdkit/discussions/6065

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom
import rdkit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# check results:
#df_sh.head()

#esomeprazole.GetNumConformers()

def getnumconf(mol):
    num = mol.GetNumConformers()
    return num

df_sh['numconf'] = df_sh.confMOL.apply(getnumconf)

# check results

df_sh['numconf'].hist(bins=100)

In [ ]:
### improve conformers

import rdkit
from rdkit.Chem import rdForceFieldHelpers
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom

import pandas as pd

def imprconf(mol):
    rdForceFieldHelpers.MMFFOptimizeMolecule(mol)
    return mol

df_sh['confMOLfinal'] = df_sh.confMOL.apply(imprconf)

df_sh.head()


In [ ]:
### get atom list

import rdkit
from rdkit import Chem

define getatseq(mol):
    mylist = mol.GetAtoms()
    return mylist

df_sh['atomseq'] = df_sh.confMOL.apply(getatseq)

In [ ]:
## cluster conformers
# Butina examples
# from: https://greglandrum.github.io/rdkit-blog/posts/2023-03-02-clustering-conformers.html
# also: https://gist.github.com/tdudgeon/b061dc67f9d879905b50118408c30aac
# https://projects.volkamerlab.org/teachopencadd/talktorials/T005_compound_clustering.html
# https://github.com/PatWalters/workshop/blob/master/clustering/taylor_butina.ipynb
# alternative: DockOnSurf

import pandas as pd
from rdkit.Chem import PandasTools, Draw
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
import seaborn as sns
from IPython.display import HTML

def cluster_conformers(mol, mode="RMSD", threshold=2.0):
	if mode == "TFD":
		dmat = TorsionFingerprints.GetTFDMatrix(mol)
	else:
		dmat = AllChem.GetConformerRMSMatrix(mol, prealigned=False)
	rms_clusters = Butina.ClusterData(dmat, mol.GetNumConformers(), threshold, isDistData=True, reordering=True)
    numclust = []
    conflist = []
    for i in len[rms_clusters]:
        for j in rms_clusters[i]:
            x = i
            numclust.append(x)
            conflist.append(j)
	return rms_clusters, conflist, numclust

df_sh['R5'] = df_sh.confMOL.apply(cluster_conformers)

df_sh[["templist","conflist","numclust"]] = df_sh.R5.to_list()

df_sh.drop("R5",axis=1,inplace=True)

In [ ]:
## write coordinates into conformers table
# https://stackoverflow.com/questions/69564484/how-to-save-rdkit-conformer-object-into-a-sdf-file

import rdkit
from rdkit import Chem
import numpy as np
import pandas as pd

def getatompos(mol):
    listoflists = []
    for i in enumerate(mol.GetConformers()):
        list2 = []
        for j in enumerate(mol.GetAtoms()):
            positions = mol.GetConformer(i).GetAtomPosition(j)
            tup = (positions.x, positions.y, positions.z)
            list2.append(tup)
        listoflists.append(list2)
    return listoflists

df_sh['atompos'] = df_sh.confMOL.apply(getatompos)

df_sh.head()

In [ ]:
### populate substituent list

import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

columns = sh_df[['atomseq', "numconf", "numclust"]]
arr2 = [tuple(r) for r in columns.to_numpy()]

sql_2 = '''
    INSERT INTO substituents (atomseq, nuconf, nuclust) 
    VALUES (%s, %s, %s)
    '''
    
val_2 = arr2

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql_2, val_2)

In [ ]:
### make conformers table

import rdkit
from rdkit import Chem
import numpy as np
import pandas as pd

df_conf = pd.DataFrame(columns=['subs_id','atomseq','atompos','clustnum','confnum'])

def getcont(dataframe1):
    subs_id = []
    for i in enumerate(dataframe1.confMOLfinal.GetConformers()):
        subs_id.append(dataframe1.id_list)
    atomseq = []
    for i in enumerate(dataframe1.confMOLfinal.GetConformers()):
        atomseq.append(dataframe1.atomseq)
    df_int = pd.DataFrame(list(zip(subs_id, atomseq, dataframe1.atompos, dataframe1.numclust, dataframe1.conflist)),columns =['subs_id','atomseq','atompos','clustnum', 'confnum'])
    df_conf.append(df_int)
    return df_conf

df_conf_f = df_sh.apply(getcont)


# check new dataframe:
df_conf.head()

print(len(df_conf.index) )


In [ ]:
### populate conformers table

columns_2 = sh_df[['subs_id','atomseq','atompos','clustnum','confnum']]
arr3 = [tuple(r) for r in columns_2.to_numpy()]

sql_3 = '''
    INSERT INTO conformers (substituent_id, atomseq, atompos, clustnum, confnum) 
    VALUES (%s, %s, %s, %s, %s)
    '''
    
val_3 = arr3

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql_1, val_1)

In [ ]:
# clean-up

import pandas
import rdkit

lst = [pd.DataFrame(), pd.DataFrame(), pd.DataFrame()]
del lst

#! conda deactivate

## switch to ase chemistry for energy computation

In [ ]:
### withdraw data from sql

# ! conda activate rdkit 

import mysql.connector as connection
import pandas as pd

pw = 
db = 'substituents'

try:
    mydb = connection.connect(host="localhost", database = db,user="root", passwd = pw,use_pure=True)
    query = "Select * from conformers;"
    result_dataFrame = pd.read_sql(query,mydb)
    mydb.close() #close the connectionexcept Exception as e:

except Exception as e:
    mydb.close()
    print(str(e))

result_dataFrame.head()

In [ ]:
### if you wish to create a simple list of tuples instead of dataframe

#def read_query(connection, query):
#    cursor = connection.cursor()
#    result = None
#    try:
#        cursor.execute(query)
#        result = cursor.fetchall()
#        return result
#    except Error as err:
#        print(f"Error: '{err}'")
        
#q1 = """
#SELECT *
#FROM teacher;
#"""

#connection = create_db_connection("localhost", "root", pw, db)
#results = read_query(connection, q1)

#for result in results:
#  print(result)

In [ ]:
### calculate point energy for each conformer

from ase import Atom
from ase.io import read
from ase.visualize import view
from ase.calculators.lj import LennardJones
from ase.optimize import BFGS
from ase.units import Hartree
import pandas as pd

import dask
import dask.dataframe as dd
from dask.distributed import progress
import time
import dask.dataframe as dd

from tqdm.notebook import tqdm
tqdm.pandas()


def calcenergy(dataframe):
    seq = dataframe.atomseq
    pos = dataframe.atompos
    a = Atoms(seq, pos)
    a.set_calculator(LennardJones())
    E_pot_emt = a.get_potential_energy()
    return E_pot_emt

ddf = dd.from_pandas(result_dataFrame,npartitions=100)

result_dataFrame["energies"] = ddf.map_partitions(calcenergy,meta=(('atomseq', 'float'), ('atompos', 'float')).compute(num_workers=4)


In [ ]:
### add values to conformers table - alternative method

columns_3 = result_dataFrame[["energies"]]
arr4 = [tuple(r) for r in columns_3.to_numpy()]

sql_4 = '''
    INSERT INTO conformers (energies) 
    VALUES (%s)
    '''
    
val_4 = arr4

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)
execute_list_query(connection, sql_4, val_4)

In [ ]:
#! conda deactivate

## find min and med energy conformers in SQL 

In [ ]:
### find member with minimum and median energy for each cluster
# https://www.sqlshack.com/sql-partition-by-clause-overview/

# absolut minimum for each substituent

sql_5 = '''
    SET substituents.lowest_energy = (SELECT conformers.substituent_id, MIN(energies) FROM conformers GROUP BY substituent_id);
    '''
    

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)

cursor = connection.cursor()
try:
    cursor.executemany(sql_5)
    connection.commit()
    print("Query successful")
except Error as err:
    print(f"Error: '{err}'")
    

In [ ]:
# add new column with cluster ID

sql_6 = '''
    ALTER TABLE conformers ADD clustID AS (substituent_id + '-' + cluster_no);
    INSERT INTO clusters.clustID
    SELECT DISTINCT clustID from conformers;
    ALTER TABLE conformers
    DROP COLUMN cluster_no;
    '''
    

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)

cursor = connection.cursor()
try:
    cursor.executemany(sql_6)
    connection.commit()
    print("Query successful")
except Error as err:
    print(f"Error: '{err}'")
    


In [ ]:
# minimum energy for each cluster

sql_7 = '''
    SET clusters.clust_le = (SELECT conformers.clustID, AVG(energies) FROM conformers GROUP BY clustID);
    '''
    

pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)

cursor = connection.cursor()
try:
    cursor.executemany(sql_7)
    connection.commit()
    print("Query successful")
except Error as err:
    print(f"Error: '{err}'")
    

In [ ]:
# average

sql_8 = '''
    SET clusters.clust_ae = (SELECT conformers.clustID, AVG(energies) FROM conformers GROUP BY clustID);
    '''
pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)

cursor = connection.cursor()
try:
    cursor.executemany(sql_7)
    connection.commit()
    print("Query successful")
except Error as err:
    print(f"Error: '{err}'")
    


In [ ]:
# median
# https://stackoverflow.com/questions/58621343/mysql-calculating-median-of-values-grouped-by-a-column

sql_9 = '''
    SET clusters.clust_me =
    (SELECT conformers.clustID, avg(conformers.energies) as clustMED
    FROM (SELECT conformers.clustID, conformers.energies, row_number() OVER(PARTITION BY clustID order by energies) rn,
    COUNT(*) OVER(PARTITION BY clustID) cnt
    FROM conformers 
    ) AS dd
    WHERE rn IN ( FLOOR((cnt + 1) / 2), FLOOR( (cnt + 2) / 2) )
    GROUP BY clustID);
    '''
pw = 
db = 'substituents'

connection = create_db_connection("localhost", "root", pw, db)

cursor = connection.cursor()
try:
    cursor.executemany(sql_7)
    connection.commit()
    print("Query successful")
except Error as err:
    print(f"Error: '{err}'")
    

In [ ]:
#! conda deactivate